In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import string
import os

In [2]:
productpath = "../../postfit_derivatives/"
tbpath = "../../fits/"

In [3]:
models_linear = ["fulllinearmodel_fit_table.csv","reducedlinearmodelNegBinom_fit_table.csv",
          "reducedlinearmodelq0_fit_table.csv","reducedlinearmodelq0ctime_fit_table.csv"]
models_nonlinear = ["nonlinearmodelq0ctime_fit_table.csv",
                    "nonlinearmodel_fit_table.csv"]

models = models_linear + models_nonlinear

# model_hash = {}
# k = -1
# for model in models:
#     k += 1
#     model_hash[model] = string.ascii_uppercase[k]

# df = pd.DataFrame.from_dict(model_hash, orient='index')
# df.to_csv('../postfit_derivatives/model_hash.csv', header=False)

In [4]:
rois = []
for model in models:
    df = pd.read_csv(tbpath + model) #get rois in all tables (some may have failed)
    rois += list(df.roi.unique())

    
rois = list(set(rois))

#get inferred
theta = df.columns[2:] 
ntheta = len(theta)



In [5]:
roi_us = np.sort([i for i in rois if i[:2]=='US'])#[::-1]
roi_other = np.sort([i for i in rois if i[:2]!='US'])#[::-1]
rois = list(roi_us) + list(roi_other)

dfwaic = pd.DataFrame(columns=['roi', 'model','linbool', 'waic'])

k = 0
for roi in rois:
#     print(roi)
    for model in models:
        k += 1
        df = pd.read_csv(tbpath + model)
        try:
            measure = df.loc[(df.roi==roi)&(df['quantile']==0.5),'waic'].values[0]
#             dfwaic.loc[k] = [roi,model_hash[model],measure]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],model in models_linear,measure]
        except:
#             dfwaic.loc[k] = [roi,model_hash[model],1e6]
            dfwaic.loc[k] = [roi,model.split('.csv')[0],model in models_linear,1e6]



In [7]:
dfwaic_sorted = dfwaic.sort_values(['waic'],ascending=True).groupby('roi')
productname = "waicsorted_topmodels_perroi.csv"
dfwaic_sorted.head(1).to_csv(productpath+productname,index=False)

In [8]:
a = dfwaic_sorted.head(1)
# print(1-a['linbool'].values[:])
x = ['linear','nonlinear']
y = [a['linbool'].values[:],1-a['linbool'].values[:]]
# sns.barplot(y=y)
print(sum(a['linbool'].values[:])/len(a['linbool'].values[:]))

0.9117647058823529
